In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#import required package
from __future__ import division
from scipy import interpolate
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import spectral.io.envi as envi
from spectral import *
import os
import re
import math
import PIL
from scipy import signal
from scipy import ndimage
from scipy import spatial
from scipy import stats
from scipy import interpolate
import traceback
import sys
import pickle
import warnings
import pandas as pd
from pprint import pprint
from sklearn.metrics import r2_score
from PIL import Image
from IPython.core.display import display, HTML
import cv2
from osgeo import gdal

In [ ]:
#file mangement generates a list containing a pth to all hdr files of pre-processed observations
def get_all_processed_HDR_addresses(parentDirectory):
    observationFiles=[]
    
    for (dirpath, dirnames, filenames) in os.walk(parentDirectory):
        observationFiles.extend(filenames)
        break
    ObservationImageAddresses=[]
    
    hdrPattern=".hdr"
    for i in observationFiles:
        if re.search(hdrPattern,i):
            ObservationImageAddresses.append(parentDirectory+'/'+i)
    return ObservationImageAddresses

In [ ]:
#read hrd files for wavlengths of all bands
def read_CRISM_Image_wavelengths(headerFileAddress):
    wavelengthFile="J:/IIT/user/Documents/CRISM Multispectral Exploration/Processed_observations/wavelength_list.txt"
    
    currentObservationHeader=envi.open(headerFileAddress)
    currentObservationImage=currentObservationHeader.open_memmap()
    currentObservationArray=np.array(currentObservationImage)
    #currentObservationArray=spectral.open_image(i)

    #read wavelengths from saved wavelength file
    with open (wavelengthFile, 'rb') as fp:
        wavelengthsOfCurrentObservation=pickle.load(fp)
    return currentObservationArray,wavelengthsOfCurrentObservation

In [ ]:
#find index of wavelngth closest to given wavelength
def find_nearest(wavelength_list,value): #find wavelength value in list closest to given value
    idx = np.searchsorted(wavelength_list, value, side="left")
    if idx > 0 and (idx == len(wavelength_list) or math.fabs(value - wavelength_list[idx-1]) < math.fabs(value - wavelength_list[idx])):
        return wavelength_list[idx-1]
    else:
        return wavelength_list[idx]

def find_wavelength_index(wavelength_list,item): #return index of in the array
    first = 0
    last = len(wavelength_list)-1
    found = False
    while( first<=last and not found):
        mid = (first + last)//2
        if wavelength_list[mid] == item :
            found = True
        else:
            if item < wavelength_list[mid]:
                last = mid - 1
            else:
                first = mid + 1	
    return mid
def find_nearest_index(wavelength_list,wavelength): #get wavelength in index closest to gicen wavelength and return its index
    (nearest_index,nearest_wavelength)=min(enumerate(wavelength_list), key=lambda x: abs(x[1]-wavelength))
    return nearest_index
def find_nearest_index_for_list(wavelength_list,wavelengths): # find index of nearest value for a list of given values
    indices=[]
    for i in wavelengths:
        indices.append(find_nearest_index(wavelength_list,i))
    return indices

In [ ]:
# given a hyperspectral datacube and wavelengths to serve as upper and lower thresholds subset the datacube along the spectral direction
def get_spectral_subset(datacube,wavelength_list,lower_threshold,upper_threshold): #returns spectral subset from lower to upper threshold ( 1.2 to 2.6 micrometer for CRISM IR)
    lower_threshold_index=find_wavelength_index(wavelength_list,lower_threshold)
    upper_threshold_index=find_wavelength_index(wavelength_list,upper_threshold)
    thresholdedDatacube=datacube[:,:,lower_threshold_index:upper_threshold_index+1]
    thresholdedWavelengthList=wavelength_list[lower_threshold_index:upper_threshold_index+1]
    return thresholdedDatacube,thresholdedWavelengthList

In [ ]:
# return an array of objects of SLI files corresponding to different mineral spectra
def getspectralObjects(spectraDirectory):
    observationFiles=[]
    for (_,_,temporaryList) in os.walk(spectraDirectory):
        observationFiles.extend(temporaryList)
        break
    hdrPattern=".hdr$"
    spectralObjects=[]
    for i in observationFiles:
        if re.search(hdrPattern,i):
            currentFile=spectraDirectory+'/'+i
            currentSpectra=envi.open(currentFile)
            spectralObjects.append(currentSpectra)
    return spectralObjects,observationFiles

In [ ]:
#given a spectral object and a wavelength list extract spectra resample them to the given wavelenght list resolution
def getResampledSpectra(spectralObject,wavelengthList):
    #read an NxM numpy array containing N spectra with M bands
    spectras=spectralObject.spectra 
    #read an array with wavelengths corresponding to each band
    sourceWavelengthList=spectralObject.bands.centers 
    #find no of bands in CRISM image
    noOfBands=np.shape(wavelengthList)[0] 
    # calculate number of spectras
    noOfSpectra=np.shape(spectras)[0]
    # create an array to store the resampled spectras
    resampledSpectras=np.zeros((noOfSpectra,noOfBands))
    #create a resampler
    resampler=BandResampler(sourceWavelengthList,wavelengthList)
    #iterate over all spectra
    for i in range(noOfSpectra):
        #resample a spectra
        resampledSpectras[i,:]=resampler(spectras[i,:])
    return resampledSpectras,spectralObject.names

In [ ]:
#given spectral object get original spectra and accompaning wavelength list
def getOriginalSpectra(spectralObject):
    #read an NxM numpy array containing N spectra with M bands
    spectras=spectralObject.spectra 
    #read an array with wavelengths corresponding to each band
    sourceWavelengthList=spectralObject.bands.centers 
    
    return spectras,spectralObject.names,sourceWavelengthList

In [ ]:
# threshold image at 2.6 micrometer
def preProcessImage(imageHeaderAddress): # given address of a CRISM header file, read the data, deal with NaN values and apply spectral subset
    datacube,wavelength_list=read_CRISM_Image_wavelengths(imageHeaderAddress) #read image and wavelength_list
    datacube=np.where(np.logical_and(datacube>0,datacube<1),datacube,0)
    lower_threshold_for_spectral_subset=1.0
    upper_threshold_for_spectral_subset=2.59 #define wavelength threshold (CRISM data too noisy post 2.6 micrometer)
    datacube,wavelength_list=get_spectral_subset(datacube,wavelength_list,lower_threshold_for_spectral_subset,upper_threshold_for_spectral_subset)
    return datacube,wavelength_list

In [ ]:
#calculate mean relfectance at wavelength with defined kernel size for datacube
def calcualteReflectaneAtWavelength(wavelength,kernelSize):
    global slicedCube
    global wavelengthList
    targetWavelength=find_nearest(wavelengthList,wavelength) #find nearest wavelength value in list
    targetWavelengthIndex=find_nearest_index(wavelengthList,wavelength) 
    BandAtWavelength=slicedCube[:,:,targetWavelengthIndex]
    for i in range(targetWavelengthIndex-math.floor(kernelSize/2),targetWavelengthIndex+math.floor(kernelSize/2)):
        BandAtWavelength=np.add(slicedCube[:,:,i],BandAtWavelength)
    BandAtWavelength=np.subtract(BandAtWavelength,slicedCube[:,:,targetWavelengthIndex])
    return BandAtWavelength/kernelSize

In [ ]:
#given a spectra apply a DoG(difference of gaussian) filter to the spectra with sigmas 7 and 4
def applyDoG(spectra,sigma1,sigma2):
    #specify the two sigmas
    #sigma1=9
    #sigma2=3
    #apply gaussian filter with the first sigma
    gauss1Filtered=ndimage.gaussian_filter1d(spectra,sigma=sigma1)
    #apply gaussian filter with the second sigma
    gauss2Filtered=ndimage.gaussian_filter1d(spectra,sigma=sigma2)
    #calculate DoG
    differenceSpectra=np.abs(np.subtract(gauss2Filtered,gauss1Filtered))
    return differenceSpectra

In [ ]:
#given the spectra and indices of minimas interpolate leniar segmaents between the minimas
def interpolateBetweenIndices(spectra,indices):
    global wavelengthList
    #create linear interpolator using wavelengths and reflectance at minima indices
    interpolator=interpolate.interp1d(wavelengthList[indices],spectra[indices],kind='linear')
    #get interpolated values at all indices
    interpolatedReflectance=interpolator(wavelengthList)
    return interpolatedReflectance

In [ ]:
#given a spectra and fraction size apply a boxcar (moving average) filter of that size to the spectra
def applyBoxcarFilter(spectra,filterFraction):
    #get number of bands in the spectra
    bands=np.shape(spectra)[0]
    #compute size of the filter
    filterSize=int(math.floor(filterFraction*bands))
    #cimpute half filter size
    halfFilterSize=int(math.floor(filterSize/2))
    #symetrically pad the spectra with half the filter size on each end
    paddedSpectra=np.pad(spectra,pad_width=halfFilterSize,mode='symmetric')
    #create an empty array to store the filtered spectra
    filteredSpectra=np.zeros_like(spectra)
    #create a counter to store filtered value
    index=0
    #iterate in original spectra range  in the padded spectra
    for i in range(halfFilterSize,halfFilterSize+bands):
        #apply the filter and store the result
        filteredSpectra[index]=np.sum(paddedSpectra[i-halfFilterSize:i+halfFilterSize])/filterSize
        #increment the counter
        index=index+1
    return filteredSpectra

In [ ]:
#given a signal and size of filter find the standard deviation signal
def getStandardDeviation(signal,windowSize):
    c1=ndimage.filters.uniform_filter(signal,windowSize, mode='reflect')
    c2=ndimage.filters.uniform_filter(signal*signal,windowSize, mode='reflect')
    return np.sqrt(c2 - c1*c1)

In [ ]:
#given a signal smmoth it adaptively
def adaptiveSmoothing(signal):
    #specify box car filter sizes 2% and 10%
    firstBoxCarSize=0.05
    secondBoxCarSize=0.15
    #specify the sigma and window fraction
    fraction1=1/15
    fraction2=1/25
    #specify the standard deviation for the diffrence of Gaussian filters
    sigma1=15
    sigma2=4
    #specify window size of standard deviation calculation
    stdWindowSize1=int(fraction1*np.shape(signal)[0])
    stdWindowSize2=int(fraction2*np.shape(signal)[0])
    #apply 2% box car filter twice
    boxcar2Spectra=applyBoxcarFilter(applyBoxcarFilter(signal,filterFraction=firstBoxCarSize),filterFraction=firstBoxCarSize)
    #apply 10% box car filter twice
    boxcar10Spectra=applyBoxcarFilter(applyBoxcarFilter(signal,filterFraction=firstBoxCarSize),filterFraction=firstBoxCarSize)
    #apply DoG on 2% box car
    dog2Spectra=applyDoG(boxcar2Spectra,sigma1,sigma2)
    #apply DoG on 10% box car
    dog10Spectra=applyDoG(boxcar10Spectra,sigma1,sigma2)
    #calculate standard deviation for both dog results
    dog2STD=getStandardDeviation(dog2Spectra,stdWindowSize1)
    dog10STD=getStandardDeviation(dog10Spectra,stdWindowSize2)
    #get smmothed spectra
    smothedSpectra=np.where(dog2STD<dog10STD,boxcar2Spectra,boxcar10Spectra)
    return smothedSpectra

In [ ]:
#given a spectra normalize it to the range (0,1)
def normalizeSpectra(spectra):
    #spectra/=np.max(np.abs(spectra),axis=0)
    #return spectra
    return (spectra - np.min(spectra))/np.ptp(spectra)

In [ ]:
#given a spectra and the indices of local minimas find the maximas for each segment demarkated by two minimas
def getMaximas(spectra,minimaIndices):
    #add the end of the spectra to the list of minimas and sort
    minimaIndices.append(len(spectra)-1)
    minimaIndices.sort()
    #create an empty list to store maximas
    maximaIndices=[]
    #iterate from 1st to the penultimate element
    for i in range(len(minimaIndices)-1):
        #extract slice of the spectra
        spectralSlice=list(spectra[minimaIndices[i]:minimaIndices[i+1]])
        #extract slice of the hull
        #append the maxima in this segment to the list of maximas
        try:
            maximaIndices.append(spectralSlice.index(max(spectralSlice))+minimaIndices[i])
        except:
            pass
    return maximaIndices

In [ ]:
#given a spectra and a hull for the spectra return pait indices marking areas of the spectra that rise aboive the hull
def getOverUnderHullSegments(spectra,hull):
    #and empty list to hold start and end index of segments over/under the hull and a flag to tell if the segment is above or below
    segmentMarkers=[]
    #subtract the hull from the input spectra
    differenceSpectra=np.subtract(spectra,hull)
    #get indices of crossover zeros in the difference spectra
    crossoverIndices=list(np.where(np.diff(np.signbit(differenceSpectra)))[0])
    #add the start and end of the spectra to the list of cross over points
    crossoverIndices.append(0)
    crossoverIndices.append(len(spectra)-1)
    #remove duplicates and sort
    crossoverIndices=list(set(crossoverIndices))
    crossoverIndices.sort()
    #iterate through segments marked by crossover points
    for i in range(len(crossoverIndices)-1):     
        #extract segment from the difference signal
        currentSegment=differenceSpectra[crossoverIndices[i]:crossoverIndices[i+1]]
        postivemagnitude=0
        negetivemagnitude=0
        #get maginutude of largest postive value in the segment
        try:
            postivemagnitude=np.max(currentSegment[np.where(currentSegment>0)[0]])
        except:
            postivemagnitude=0
        #get maginutude of largest postive value in the segment
        try:
            negetivemagnitude=abs(np.min(currentSegment[np.where(currentSegment<0)[0]]))
        except:
            negetivemagnitude=0    
        #calculate sum of the segment check if it is positve or negetive and add to the list
        if postivemagnitude>negetivemagnitude:
            segmentMarkers.append([crossoverIndices[i],crossoverIndices[i+1],1])
        else:
            segmentMarkers.append([crossoverIndices[i],crossoverIndices[i+1],-1])
    return differenceSpectra,segmentMarkers

In [ ]:
#given a spectra,set of initial minima Points and an initial hull iteratively improve upon it
def generateIdealHull(spectra,initialHull,minimaIndices,hullIndices):
    #get markers for segments above or below the hull
    differenceSpectra,segmentMarkers=getOverUnderHullSegments(spectra,initialHull)
    for i in segmentMarkers:
        #get starting index of segment
        startIndex=i[0]
        #get ending index of segment
        endIndex=i[1]
        #over or under
        overUnderFlag=i[2]
        #if the flag is over add the maxima of that segment to the list of hull points and cross over points to minima indices
        if overUnderFlag==1:
            hullIndices.append(np.argmax(differenceSpectra[startIndex:endIndex])+startIndex)
            minimaIndices.append(startIndex)
            minimaIndices.append(endIndex)
    
    #remove duplicates from hull indices and minima indices and sort
    hullIndices=list(set(hullIndices))
    hullIndices.sort()
    minimaIndices=list(set(minimaIndices))
    minimaIndices.sort()
    #interpolate between maximas to get the hull
    hull=interpolateBetweenIndices(spectra,hullIndices)
    return hullIndices,hull,minimaIndices

In [ ]:
#given a spectra get hull points based on an alternate method
def getInitialHull(spectra):
    #apply  box car filter twice
    smoothedSpectra=adaptiveSmoothing(spectra)
    
    #calculate indices of minima for both versions of smoothed spectra
    minimaIndices=list(signal.argrelextrema(smoothedSpectra,np.less)[0])
    
    #calculate indices of maximas for smoothed spectra
    hullIndices=getMaximas(spectra,minimaIndices)
    
    #append the first and last point in the spectra to close the hull
    hullIndices.append(0)
    hullIndices.append(len(spectra)-1)
    hullIndices.sort()
    
    #interpolate between maximas to get the hull
    hull=interpolateBetweenIndices(spectra,hullIndices)
    
    return hullIndices,hull,minimaIndices

In [ ]:
#given a spectra and indices of its hull points check if 2 or more hull points are between 2.1 and 2.4 micrometer
#if find all hull points with low smoothing ahead of the index corresponding to the hull point closest to 2.1 micrometer
def getOhHullPoints(crSpectra,hullPoints):
    #specify check range
    lowerWavelengthBound=2.1
    upperWavelengthBound=2.5
    #get wavlengths corresponding to hull points
    global wavelengthList
    hullPointWavelengths=wavelengthList[hullPoints]
    #get indices of hull points in the wavelength bound
    boundedIndices=np.array(hullPoints)[np.where(np.logical_and(hullPointWavelengths>=lowerWavelengthBound,hullPointWavelengths<=upperWavelengthBound))[0].tolist()]
    
    #check if there is only 1 elements if so return the original list of hull points
    if len(boundedIndices)<2:
        return hullPoints
    #calculate diffence between succesive indices
    indicesDifference=np.absolute(np.subtract(boundedIndices[:-1],boundedIndices[1:])).tolist()
    #check if all consecitive indices whose difference is less than 10 and return original list of hull points
    if all(i <10 for i in indicesDifference):
        return hullPoints
    
    #get the index for the first hull point in bounds
    breakPointIndex=boundedIndices[0]
    #extract portion of the continuum removed spectra after the breakPointIndex
    spectraSegment=applyBoxcarFilter(crSpectra,0.02)[breakPointIndex:]
    
    #get maxima indices
    maximaIndices=list(signal.argrelextrema(spectraSegment,np.greater)[0])
    return maximaIndices+breakPointIndex

In [ ]:
#given a continuum removed spectra remove any segments that are less than 6 points wide or whose average value is more than 0.99
def removeNonSegments(crSpectra):
    #create an array to hold processed spectra
    processedSpectra=np.ones_like(crSpectra)
    #get hull point indices i.e where continuum removed spectra equals 1
    hullPointIndices=np.where(crSpectra==1)[0]
    #iterate between consecutive hull points
    for i in range(len(hullPointIndices)-1):
        #extract spectral slice between current pair of hull points
        currentSpectralSlice=crSpectra[hullPointIndices[i]:hullPointIndices[i+1]]
        #calculate length of current spectral slice
        sliceLength=np.shape(currentSpectralSlice)[0]
        #if length of the slice is more than 5 and mean is less than 0.95 keep the slice
        if sliceLength>5 and np.min(currentSpectralSlice)<0.95:
            processedSpectra[hullPointIndices[i]:hullPointIndices[i+1]]=currentSpectralSlice
    return processedSpectra

In [ ]:
#given a continuum removed spectra identify maximas between consecutive hull points and set the maximum maxima whose value is also greater than an absolute threshold to be a hull points
def finalAdjustments(crSpectra,hullIndices):
    #set points in the continuum removed spectra at hull indices equal to 1
    crSpectra[hullIndices]=1.0
    #remove any segments too small to be absorptions features
    crSpectra=removeNonSegments(crSpectra)
    #set continuum removed spectra points equal to 1 to be hull points
    hullIndices=np.where(crSpectra==1)[0]
    #set points in the continuum removed spectra at hull indices equal to 1
    crSpectra[hullIndices]=1.0
    #remove any segments too small to be absorptions features
    crSpectra=removeNonSegments(crSpectra)
    #set continuum removed spectra points equal to 1 to be hull points
    hullIndices=np.where(crSpectra==1)[0]
    return crSpectra,hullIndices

In [ ]:
#given a spectra calculate the hull points and hull for using 2 different levels of smoothing and plot the results
def plotHull(normalizedSpectra):
    #get hull indices for both levels of smoothing
    hullIndices,hull,minimaIndices=getInitialHull(normalizedSpectra)
    #iteratively improve both hulls
    while(True):
        #save hulls from previous iterations
        prevHull=hull
        #improve the hull
        hullIndices,hull,minimaIndices=generateIdealHull(normalizedSpectra,hull,minimaIndices,hullIndices)
        #if no improvements occoured in the hull exit
        if (prevHull==hull).all():
            break
    #find any points that rise above the continuum
    hullAdditionalPoints=np.where(np.divide(normalizedSpectra,hull)>1)[0]
    #add those points to hull points
    hullIndices.extend(hullAdditionalPoints)
    #remove any duplicates and sort
    hullIndices=list(set(hullIndices))
    hullIndices.sort()
    #interpolate between points to get final hull
    hull=interpolateBetweenIndices(normalizedSpectra,hullIndices)
    
    #remove the continuum
    cr=np.ones_like(normalizedSpectra)
    cr=np.divide(normalizedSpectra,hull,out=cr,where=hull!=0)
    #get OH hull points
    hullIndices.extend(getOhHullPoints(cr,hullIndices))
    #interpolate a hull
    hull=interpolateBetweenIndices(normalizedSpectra,hullIndices)
    
    #find any points that rise above the continuum
    hullAdditionalPoints=np.where(np.divide(normalizedSpectra,hull)>1)[0]
    #add those points to hull points
    hullIndices.extend(hullAdditionalPoints)
    #remove any duplicates and sort
    hullIndices=list(set(hullIndices))
    hullIndices.sort()
    #interpolate between points to get final hull
    hull=interpolateBetweenIndices(normalizedSpectra,hullIndices)
    
    #remove the continuum
    cr=np.ones_like(normalizedSpectra)
    cr=np.divide(normalizedSpectra,hull,out=cr,where=hull!=0)
    #make final adjustments to the continuum removed spectra
    cr,hullIndices=finalAdjustments(cr,hullIndices)
    return cr

In [ ]:
#given a spectra calculate the hull points and hull for using 2 different levels of smoothing and plot the results
def plotHullnCR(normalizedSpectra):
    #get hull indices for both levels of smoothing
    hullIndices,hull,minimaIndices=getInitialHull(normalizedSpectra)
    #iteratively improve both hulls
    while(True):
        #save hulls from previous iterations
        prevHull=hull
        #improve the hull
        hullIndices,hull,minimaIndices=generateIdealHull(normalizedSpectra,hull,minimaIndices,hullIndices)
        #if no improvements occoured in the hull exit
        if (prevHull==hull).all():
            break
    #find any points that rise above the continuum
    hullAdditionalPoints=np.where(np.divide(normalizedSpectra,hull)>1)[0]
    #add those points to hull points
    hullIndices.extend(hullAdditionalPoints)
    #remove any duplicates and sort
    hullIndices=list(set(hullIndices))
    hullIndices.sort()
    #interpolate between points to get final hull
    hull=interpolateBetweenIndices(normalizedSpectra,hullIndices)
    
    #remove the continuum
    cr=np.ones_like(normalizedSpectra)
    cr=np.divide(normalizedSpectra,hull,out=cr,where=hull!=0)
    #get OH hull points
    hullIndices.extend(getOhHullPoints(cr,hullIndices))
    #interpolate a hull
    hull=interpolateBetweenIndices(normalizedSpectra,hullIndices)
    
    #find any points that rise above the continuum
    hullAdditionalPoints=np.where(np.divide(normalizedSpectra,hull)>1)[0]
    #add those points to hull points
    hullIndices.extend(hullAdditionalPoints)
    #remove any duplicates and sort
    hullIndices=list(set(hullIndices))
    hullIndices.sort()
    #interpolate between points to get final hull
    hull=interpolateBetweenIndices(normalizedSpectra,hullIndices)
    
    #remove the continuum
    cr=np.ones_like(normalizedSpectra)
    cr=np.divide(normalizedSpectra,hull,out=cr,where=hull!=0)
    #make final adjustments to the continuum removed spectra
    cr,hullIndices=finalAdjustments(cr,hullIndices)
    return cr,hull

In [ ]:
#given a datacube return its footprint i.e pixels in and out of the scene
def getDatacubeFootprint(datacube):
    [row,col,bands]=np.shape(datacube)
    #create an array to hold footprint of the image
    footprint=np.zeros_like(datacube[:,:,0])
    #get foot print of in scene pixels
    for i in range(row):
        for j in range(col):
            if 0==datacube[i,j,:].all():
                footprint[i,j]=1
    return footprint

In [ ]:
#given a spectra normalize it to the range (0,1)
def normalizeSpectra(spectra):
    upperLimit=1
    lowerLimit=0.0000001
    maximumValue=np.amax(spectra)
    minimumValue=np.amin(spectra)
    normalizedSpectra=(upperLimit-lowerLimit)*(spectra-minimumValue)/(maximumValue-minimumValue)+lowerLimit
    return normalizedSpectra

In [ ]:
#given a continuum removed spectra get the start and ending points of all absorption features
def getHullPoints(crSpectra):
    #create a empty lists to store hull
    hullPoints=[]
    #get indices for all 1s n the continuum removed spectra
    onePoints=list(np.where(crSpectra==1)[0])
    for i in range(len(onePoints)-1):
        if onePoints[i]!=onePoints[i+1]-1 or onePoints[i-1]+1!=onePoints[i]:
            hullPoints.append(onePoints[i])
    #add starting and ending point of spectra
    hullPoints.append(0)
    hullPoints.append(np.shape(crSpectra)[0]-1)
    #remove any duplicates and sort
    hullPoints=list(set(hullPoints))
    hullPoints.sort()
    
    #create 2 empty lists to hold starting and ending points of absorption features
    startPoints=[]
    endPoints=[]
    for i in range(len(hullPoints)-1):
        #get a spectral slice between the ith and i+1th hull point
        spectralSlice=crSpectra[hullPoints[i]:hullPoints[i+1]]
        #if mean of spectral slice is more than 0.99 it is not an absorption feature
        if np.mean(spectralSlice)<0.99:
            startPoints.append(hullPoints[i])
            endPoints.append(hullPoints[i+1])
    return startPoints,endPoints

In [ ]:
#given a continuum removed spectra extract each spectral feature and return 2 lists one of the spectra features and the other wavelengths for each feature
def getAbsorptionFeatures(crSpectra):
    global wavelengthList
    startPoints,endPoints=getHullPoints(crSpectra)
    #create two empty lists to hold reflectances and wavelengths for absorption features
    features=[]
    wavelengths=[]
    for i in range(len(startPoints)):
        featureReflectance=crSpectra[startPoints[i]:endPoints[i]+1]
        featureWavelengths=wavelengthList[startPoints[i]:endPoints[i]+1]
        features.append(featureReflectance)
        wavelengths.append(featureWavelengths)
    return features,wavelengths

In [ ]:
#given a absorption feature get its parameters
def getAbsorbanceParametersPolynomial(currentWavelength,currentReflectance):
    #set number of points to interpolate on each side
    noOfPoints=20
    
    #get band centre as band minima
    bandCentre=currentWavelength[np.argmin(currentReflectance)]
    #get band depth
    bandDepth=np.max(currentReflectance)-currentReflectance[np.argmin(currentReflectance)]
    
    #create array to hold left half of the feature
    leftFeaturePoints=[]
    #create array to hold right half of the feature
    rightFeaturePoints=[]
    #create array to hold left half of wavelengths
    leftWavelengthPoints=[]
    #create array to hold right half of wavelengths
    rightWavelengthPoints=[]
    
    #check if there are set no. of points to the left of the band minima and extract 10 points to the left, if less than 10 extract all points
    if np.argmin(currentReflectance)>noOfPoints:
        leftFeaturePoints=currentReflectance[np.argmin(currentReflectance)-noOfPoints:np.argmin(currentReflectance)]
        leftWavelengthPoints=currentWavelength[np.argmin(currentReflectance)-noOfPoints:np.argmin(currentReflectance)]
    else:
        leftFeaturePoints=currentReflectance[:np.argmin(currentReflectance)]
        leftWavelengthPoints=currentWavelength[:np.argmin(currentReflectance)]
    #check if there are set no. of points to the right of the band minima and extract 10 points to the right, if less than 10 extract all points
    if np.argmin(currentReflectance)-np.shape(currentReflectance)[0]>noOfPoints:
        rightFeaturePoints=currentReflectance[np.argmin(currentReflectance):np.argmin(currentReflectance)-noOfPoints]
        rightWavelengthPoints=currentWavlength[np.argmin(currentReflectance):np.argmin(currentReflectance)-noOfPoints]
    else:
        rightFeaturePoints=currentReflectance[np.argmin(currentReflectance):]
        rightWavelengthPoints=currentWavelength[np.argmin(currentReflectance):]
    
    #combine the left and right lists
    wavelength=np.concatenate((leftWavelengthPoints,rightWavelengthPoints))
    reflectance=np.concatenate((leftFeaturePoints,rightFeaturePoints))
        
    #calculate full-width-half-maxima
    halfFeatureLeftIndex=find_nearest_index(currentReflectance[:np.argmin(currentReflectance)],1-bandDepth/2)
    halfFeatureRightIndex=find_nearest_index(currentReflectance[np.argmin(currentReflectance):],1-bandDepth/2)+np.argmin(currentReflectance)
    
    #get step size as 1/4 of the difference between consecutive bands
    bandWidth=abs(wavelength[0]-wavelength[1])
    stepSize=bandWidth/4
    #generate list of interpolated wavelengths
    interpolatedWavelengths=np.arange(wavelength[0],wavelength[-1],stepSize)
    #fit a polynomial to the data
    polynomial=np.poly1d(np.polyfit(wavelength,reflectance,4))
    polynomialReflectance=polynomial(interpolatedWavelengths)
    #get band centre
    bandCentre=currentWavelength[np.argmin(currentReflectance)]
    #get band depth
    bandDepth=1-polynomial(bandCentre)
    if bandDepth>1:
        bandDepth=1
    fwhm=abs(currentWavelength[halfFeatureLeftIndex]-currentWavelength[halfFeatureRightIndex])
    return bandCentre,bandDepth,fwhm,polynomialReflectance,interpolatedWavelengths

In [ ]:
#given a half feature interpolate the feature to triple the number of points
def interpolateLowerFeature(lowerWavelength,lowerRelfectance):
    #get step size as 1/5 of the difference between consecutive bands
    bandWidth=abs(lowerWavelength[0]-lowerWavelength[1])
    stepSize=bandWidth/5
    #generate list of interpolated wavelengths
    interpolatedWavelengths=np.arange(lowerWavelength[0],lowerWavelength[-1],stepSize)
    #fit a polynomial to the data
    interpolator=interpolate.interp1d(lowerWavelength,lowerRelfectance,kind='cubic')
    interpolatedReflectance=interpolator(interpolatedWavelengths)
    return interpolatedWavelengths,interpolatedReflectance

In [ ]:
#calculate absorption peaks for all features
def getFeatureParameters(wavelengths,reflectances):
    #create empty lists to hold band centres,band depths,fwhm, and left & right shoulders
    bandCentres=[]
    bandDepths=[]
    fwhms=[]
    leftShoulders=[]
    rightShoulders=[]
    bandMinimas=[]
    #iterate over each feature

    for i in range(len(reflectances)):
        
        currentWavelength=wavelengths[i]
        currentReflectance=reflectances[i]
        #interpolate the feature if not possible use the original feature
        try:
            currentWavelength,currentReflectance=interpolateLowerFeature(currentWavelength,currentReflectance)
        except:
            pass
        #get lower half of the feature
        #get band centre
        bandCentre=currentWavelength[np.argmin(currentReflectance)]
        #get band depth
        bandDepth=np.max(currentReflectance)-currentReflectance[np.argmin(currentReflectance)]
        #calculate full-width-half-maxima
        halfFeatureLeftIndex=find_nearest_index(currentReflectance[:np.argmin(currentReflectance)],1-bandDepth/2)
        halfFeatureRightIndex=find_nearest_index(currentReflectance[np.argmin(currentReflectance):],1-bandDepth/2)+np.argmin(currentReflectance)
        #extract half wavelength and reflectance
        halfWavelength=currentWavelength[halfFeatureLeftIndex:halfFeatureRightIndex]
        halfReflectance=currentReflectance[halfFeatureLeftIndex:halfFeatureRightIndex]
        
        
        #get parameters for polynomial fit
        featureBandCentre,featureBandDepth,fwhm,fittedReflectance,fittedWavelength=getAbsorbanceParametersPolynomial(currentWavelength,currentReflectance)
        
        
        #append parameters
        try:
            bandCentres.append(featureBandCentre[0])
        except:
            bandCentres.append(featureBandCentre)
        try:
            bandDepths.append(featureBandDepth[0])
        except:
            bandDepths.append(featureBandDepth)
        #bandDepths.append(featureBandDepth)
        fwhms.append(fwhm)
        leftShoulders.append(wavelengths[i][0])
        rightShoulders.append(wavelengths[i][-1])
    #return featureBandCentre,featureBandDepth,fwhm,fittedReflectance,fittedWavelength,bandMinimas
    return bandCentres,bandDepths,fwhms,leftShoulders,rightShoulders

In [ ]:
#given lists corresponding to spectral parameters for n features of a sample spectra. Check, if the sample is a carbonate
def checkForCarbonate(bandCentres,bandDepths,fwhms,leftShoulders,rightShoulders):
    #specify range of acceptable values
    bandCentre23LowerLimit=2.285
    bandCentre23UpperLimit=2.345
    bandCentre25LowerLimit=2.485
    bandCentre25UpperLimit=2.545
    interBandGapLowerLimit=0.185
    interBandGapUpperLimit=0.215
    
    #specify ideal band centre values for Fe and Mg carbonates
    feFirstIdealBandCentre=2.33
    feSecondIdealBandCentre=2.53
    mgFirstIdealBandCentre=2.30
    mgSecondIdealBandCentre=2.50
    
    #create a flag to check if the sample is carbonate
    carbonateFlag=False
    #variable to store index of 2.3 feature
    firstCO3FeatureIndex=-1
    #get no of features
    noOfFeatures=len(fwhms)
    
    #iterate over all features
    for i in range(noOfFeatures):
        if i<noOfFeatures-1 and bandCentres[i]<=bandCentre23UpperLimit and bandCentres[i]>=bandCentre23LowerLimit and bandCentres[i+1]<=bandCentre25UpperLimit and bandCentres[i+1]>=bandCentre25LowerLimit and abs(bandCentres[i]-bandCentres[i+1])<=interBandGapUpperLimit and abs(bandCentres[i]-bandCentres[i+1])>=interBandGapLowerLimit:
            firstCO3FeatureIndex=i
            carbonateFlag=True
    
    #if sample is carbonate classify as Iron/Calcium or Magnesium
    if carbonateFlag:
        #calculate distances from ideal Iron band centres
        fe23Distance=abs(feFirstIdealBandCentre-bandCentres[firstCO3FeatureIndex])
        fe25Distance=abs(feSecondIdealBandCentre-bandCentres[firstCO3FeatureIndex+1])
        #record greater distance
        feDistanceFromIdeal=max(fe23Distance,fe25Distance)
        #calcualte distances from ideal Magnesium band centres
        mg23Distance=abs(mgFirstIdealBandCentre-bandCentres[firstCO3FeatureIndex])
        mg25Distance=abs(mgSecondIdealBandCentre-bandCentres[firstCO3FeatureIndex+1])
        #record greater distance
        mgDistanceFromIdeal=max(mg23Distance,mg25Distance)
        #check which recorded distance is lesser, mark as that carbonate
        if mgDistanceFromIdeal<feDistanceFromIdeal:
            return 1 #flag for Mg carbonte
        else:
            return 2 #flag for Fe carbonte
    return 0 #flag for non carboante

In [ ]:
#given lists corresponding to band centres, band depth and fwhms of all features in a spectra find  1.4 and 1.9 mircon feature for phyllosilicates check and classify the sample as Phyllosilicate or not
def checkPhyllosilicate(bandCentres,bandDepths,fwhms):
    #specify acceptable ranges
    bandCentre14LowerLimit=1.33
    bandCentre14UpperLimit=1.44
    bandCentre19LowerLimit=1.89
    bandCentre19UpperLimit=1.96
    fwhm14LowerLimit=0.019
    fwhm14UpperLimit=0.263
    fwhm19LowerLimit=0.02
    fwhm19UpperLimit=0.15
    # create variables to hold indices for relavent features
    feature14Index=-1
    feature19Index=-1
    for i in range(len(bandCentres)):
        #check for 1.9 micron feature
        if bandCentres[i]<=bandCentre14UpperLimit and bandCentres[i]>=bandCentre14LowerLimit and fwhms[i]<=fwhm14UpperLimit and fwhms[i]>=fwhm14LowerLimit and feature14Index==-1:
            print("First phyllo. feature at index: "+str(i))
            feature14Index=i
        #check for 1.9 micron feature
        elif bandCentres[i]<=bandCentre19UpperLimit and bandCentres[i]>=bandCentre19LowerLimit and fwhms[i]<=fwhm19UpperLimit and fwhms[i]>=fwhm19LowerLimit and feature19Index==-1:
            print("First phyllo. feature at index: "+str(i))
            feature19Index=i
    #check if both features found if so return True
    if feature14Index>=0 and feature19Index>=0:
        return True
    return False

In [ ]:
#given a preprocessed and normalized spectra and its name process it and classify as Carbonate type or not carbonate
def processSpectra(spectra):
    #set minimum width of feature
    minimumFeatureWidth=0.08
    cr=plotHull(spectra)
    features,wavelengths=getAbsorptionFeatures(cr)
    tempFeat=[]
    tempWavelengths=[]
    for i in range(len(wavelengths)):
        if wavelengths[i][-1]-wavelengths[i][0]>minimumFeatureWidth:
            tempFeat.append(features[i])
            tempWavelengths.append(wavelengths[i])
    features=tempFeat
    wavelengths=tempWavelengths
    bandCentres,bandDepths,fwhms,leftShoulders,rightShoulders=getFeatureParameters(wavelengths,features)
    
    #check carbonate flag
    co3Flag=checkForCarbonate(bandCentres,bandDepths,fwhms,leftShoulders,rightShoulders)
    if co3Flag==0:
        #print("Not CO3")
        return 0
    elif co3Flag==1:
        #print("MgCO3")
        return 1
    else:
        #print("FeCO3")
        return 2

In [ ]:
#given a datacube and wavelength list process each spectra and and generate a map of Fe and Mg CO3 deposits
def generateCO3Map(datacube,wavelengthList,imageFootprint):
    #intitalize variables
    nonCO3=0
    feCO3=0
    mgCO3=0
    totalSpectra=0
    #initialize lists to hold co3 pixel indices
    mgCO3Rows=[]
    mgCO3Cols=[]
    feCO3Rows=[]
    feCO3Cols=[]
    #get dimensions of image
    [rows,cols,bands]=np.shape(datacube)
    #create an array to hold deposit locations
    co3Map=np.zeros_like(datacube[:,:,0])
    #iterate over all pixels
    for i in range(rows):
        for j in range(cols):
            #print(f"{i}-{j} pixels processed")
            if imageFootprint[i,j]==-1 or np.any(datacube[i,j,:]==np.nan) or np.any(datacube[i,j,:]==65535.0) or np.all(datacube[i,j,:]==datacube[i,j,0]):
                co3Map[i,j]=-1
                continue
            else:
                carbonateFlag=0
                #get the spectra
                spectra=datacube[i,j,:]
                #normalize spectra
                spectra=normalizeSpectra(spectra)

                #process the spectra
                carbonateFlag=processSpectra(spectra)
                co3Map[i,j]=carbonateFlag
                if carbonateFlag==1:
                    mgCO3=mgCO3+1
                    mgCO3Rows.append(i)
                    mgCO3Cols.append(j)
                elif carbonateFlag==2:
                    feCO3=feCO3+1
                    feCO3Rows.append(i)
                    feCO3Cols.append(j)
                else:
                    nonCO3=nonCO3+1
            if (i*cols+j)%1000==0:
                print(f"{i*rows+j} pixels processed")
            
    print(f"Number of total spectra---{totalSpectra}")
    print(f"Number of nonCO3 spectra---{nonCO3}")
    print(f"Number of MgCO3 spectra---{mgCO3}")
    print(f"Number of FeCO3 spectra---{feCO3}")
    print(f"\t\t")
    return co3Map,feCO3Rows,feCO3Cols,mgCO3Rows,mgCO3Cols

In [ ]:
#calculate band depth based indices for Fe and Ca/Mg rich carbonates
def calculateCarbonateBandDepthIndices(datacube,wavelength_list):
    R2345Wavelength=2.345 #define constant values to calculate indices, wavelengths, and kernel size 
    R2250Wavelength=2.250
    R2430Wavelength=2.430
    R2537Wavelength=2.537
    R2602Wavelength=2.590#2.602

    R2345KernelSize=5
    R2250KernelSize=5
    R2430KernelSize=5
    R2602KernelSize=1
    R2537KernelSize=5

    R2295Wavelength=2.295
    R2165Wavelength=2.165
    R2364Wavelength=2.364
    R2480Wavelength=2.480
    R2570Wavelength=2.570

    R2295KernelSize=5
    R2165KernelSize=5
    R2364KernelSize=5
    R2480KernelSize=5
    R2570KernelSize=5


    bForR2430=(R2345Wavelength-R2250Wavelength)/(R2430Wavelength-R2250Wavelength)
    aForR2250=1-bForR2430
    bForR2602=(R2537Wavelength-R2430Wavelength)/(R2602Wavelength-R2430Wavelength)
    aForR2430=1-bForR2602

    bForR2364=(R2295Wavelength-R2165Wavelength)/(R2364Wavelength-R2165Wavelength)
    aForR2165=1-bForR2364
    bForR2570=(R2480Wavelength-R2364Wavelength)/(R2570Wavelength-R2364Wavelength)
    aForR2364=1-bForR2570
    
    
    R2295Reflectance=calcualteReflectaneAtWavelength(R2295Wavelength,R2295KernelSize) #read bands corresponding to wavelengths
    R2165Reflectance=calcualteReflectaneAtWavelength(R2165Wavelength,R2165KernelSize)
    R2364Reflectance=calcualteReflectaneAtWavelength(R2364Wavelength,R2364KernelSize)
    R2480Reflectance=calcualteReflectaneAtWavelength(R2480Wavelength,R2480KernelSize)
    R2570Reflectance=calcualteReflectaneAtWavelength(R2570Wavelength,R2570KernelSize)

    temp1Den=np.add(aForR2165*R2165Reflectance,bForR2364*R2364Reflectance) # calculation for MIN2295_2480 minimum band depth at 2.3 and 2.5 micrometer 
    temp2Den=np.add(aForR2364*R2364Reflectance,bForR2570*R2570Reflectance)
    temp3=np.full_like(R2570Reflectance,2.0)
    
    temp1=np.divide(R2295Reflectance,temp1Den,temp3,where=np.logical_and(temp2Den>0,temp2Den<1))
    temp1=np.subtract(np.full_like(R2570Reflectance,1),temp1)
    temp3=np.full_like(R2570Reflectance,2.0)
    temp2=np.divide(R2480Reflectance,temp2Den,temp3,where=np.logical_and(temp2Den>0,temp2Den<1))
    temp2=np.subtract(np.full_like(R2570Reflectance,1),temp2)
    MIN2295_2480=np.minimum(temp1,temp2)
    
    R2345Reflectance=calcualteReflectaneAtWavelength(R2345Wavelength,R2345KernelSize) #read bands corresponding to wavelengths
    R2250Reflectance=calcualteReflectaneAtWavelength(R2250Wavelength,R2250KernelSize)
    R2430Reflectance=calcualteReflectaneAtWavelength(R2430Wavelength,R2430KernelSize)
    R2537Reflectance=calcualteReflectaneAtWavelength(R2537Wavelength,R2537KernelSize)
    R2602Reflectance=calcualteReflectaneAtWavelength(R2602Wavelength,R2602KernelSize)
    
    temp1Den=np.add(aForR2250*R2250Reflectance,bForR2430*R2430Reflectance) # calculation for MIN2345_2537 minimum band depth at 2.33 and 2.53 micrometer
    temp2Den=np.add(aForR2430*R2250Reflectance,bForR2602*R2602Reflectance)
    temp3=np.full_like(R2345Reflectance,2.0)
    
    temp1=np.divide(R2345Reflectance,temp1Den,temp3,where=np.logical_and(temp2Den>0,temp2Den<1))
    temp1=np.subtract(np.full_like(R2345Reflectance,1),temp1)
    temp3=np.full_like(R2345Reflectance,2.0)
    temp2=np.divide(R2537Reflectance,temp2Den,temp3,where=np.logical_and(temp2Den>0,temp2Den<1))
    temp2=np.subtract(np.full_like(R2345Reflectance,1),temp2)
    MIN2345_2537=np.minimum(temp1,temp2)
    
    return MIN2295_2480,MIN2345_2537

In [ ]:
# given row number and column number get coordinates of the pixel
def getCoordinates(row,col):
    global ds
    c,a,b,f,d,e=ds.GetGeoTransform()
    """Returns global coordinates to pixel center using base-0 raster index"""
    xp=a*col+b*row+a*0.5+b*0.5+c
    yp=d*col+e*row+d*0.5+e*0.5+f
    return xp,yp

In [ ]:
#calculate mean relfectance at wavelength with defined kernel size 
def calcualteReflectaneAtWavelength(wavelength,kernelSize):
    global slicedCube
    global wavelengthList
    targetWavelength=find_nearest(wavelengthList,wavelength) #find nearest wavelength value in list
    targetWavelengthIndex=find_nearest_index(wavelengthList,wavelength) 
    BandAtWavelength=slicedCube[:,:,targetWavelengthIndex]
    for i in range(targetWavelengthIndex-math.floor(kernelSize/2),targetWavelengthIndex+math.floor(kernelSize/2)):
        BandAtWavelength=np.add(slicedCube[:,:,i],BandAtWavelength)
    BandAtWavelength=np.subtract(BandAtWavelength,slicedCube[:,:,targetWavelengthIndex])
    return BandAtWavelength/kernelSize

In [ ]:
headerFileAddress="G:/000093BE/FRT000093BE_07_IF166L_TRRB_sabcondpub_v1_b6_nr_ds.hdr"
headerFileAddress="G:/0002fe75/frs0002fe75_01_if168l_trr3_CAT_corr_denoised_p.hdr"
print(headerFileAddress)
currentObservationHeader=envi.open(headerFileAddress)
currentObservationImage=currentObservationHeader.open_memmap()
datacube=np.array(currentObservationImage)
del currentObservationImage
wavelengthList=np.array(currentObservationHeader.bands.centers)

In [ ]:
#set the wavelength of the last band to retain
finalBandWavelength=2.60212
#subset the datacube and wavelength list at the final wavelength
datacube=datacube[:,:,np.logical_and(wavelengthList<=2.60212,wavelengthList>=1.05375)]
wavelengthList=wavelengthList[np.logical_and(wavelengthList<=2.60212,wavelengthList>=1.05375)]

In [ ]:
slicedCube=datacube
#identify in scene pixels and create a map where all inscene pixels are 1 and out of scene pixels are -1
imageFootprint=np.full_like(datacube[:,:,0],1)
datacube[np.where(np.isnan(datacube))]=65535.0
[rows,cols,bands]=np.shape(datacube)
for i in range(rows):
    for j in range(cols):
        temp=datacube[i,j,:]
        if np.any(temp==65535.0):
            imageFootprint[i,j]=-1
datacube[np.where(np.isnan(datacube))]=65535.0

In [ ]:
print(datacube.shape)
print(wavelengthList.shape)

In [ ]:
datacube[144,99,:]

In [ ]:
plt.imshow(imageFootprint)
plt.grid()

In [ ]:
#get carbonate map 
co3Map,feCO3Rows,feCO3Cols,mgCO3Rows,mgCO3Cols=generateCO3Map(datacube,wavelengthList,imageFootprint)

In [ ]:
np.save("G://0002fe75/frs0002fe75_01_if168l_trr3_CAT_corr_denoised_p_co3.npy",
           co3Map,
           allow_pickle=True,
           fix_imports=True)

In [ ]:
#calculate spectral parameters
MIN2295_2480,MIN2345_2537=calculateCarbonateBandDepthIndices(datacube,wavelengthList)
#threshold the arrays
MIN2295_2480=np.where(MIN2295_2480>0.03,MIN2295_2480,np.nan)
MIN2345_2537=np.where(MIN2345_2537>0.03,MIN2345_2537,np.nan)
MIN2295_2480=np.where(imageFootprint>0,MIN2295_2480,np.nan)
MIN2345_2537=np.where(imageFootprint>0,MIN2345_2537,np.nan)
#get unified map of standard parameters
originalCO3Map=np.maximum(MIN2295_2480,MIN2345_2537)
originalCO3Map=np.where(imageFootprint>0,originalCO3Map,np.nan)
#get carbonate map
myMap=np.where(co3Map>0,1,0)
#myMap=ndimage.median_filter(myMap,size=2)
#myMap=ndimage.morphology.binary_erosion(myMap,structure=np.ones((2,2)).astype(myMap.dtype),iterations=1)
#myMap=ndimage.morphology.binary_dilation(myMap,structure=np.ones((2,2)).astype(myMap.dtype),iterations=1)
bothMap=np.where(myMap==1,myMap,np.nan)
#get fe carbonate map
myMap=np.where(co3Map==2,1,0)
#myMap=ndimage.median_filter(myMap,size=2)
#myMap=ndimage.morphology.binary_erosion(myMap,structure=np.ones((2,2)).astype(myMap.dtype),iterations=1)
#myMap=ndimage.morphology.binary_dilation(myMap,structure=np.ones((2,2)).astype(myMap.dtype),iterations=1)
feMap=np.where(myMap==2,1,np.nan)
#get mg carbonate map
myMap=np.where(co3Map==1,1,np.nan)
#myMap=ndimage.median_filter(myMap,size=2)
#myMap=ndimage.morphology.binary_erosion(myMap,structure=np.ones((2,2)).astype(myMap.dtype),iterations=1)
#myMap=ndimage.morphology.binary_dilation(myMap,structure=np.ones((2,2)).astype(myMap.dtype),iterations=1)
mgMap=np.where(myMap==1,1,np.nan)



In [ ]:
'''headerFileAddress="G:/000093BE/FRT000093BE_07_IF166L_TRRB_sabcondpub_v1_b6_nr_ds_p_2014params.hdr"
print(headerFileAddress)
currentObservationHeader=envi.open(headerFileAddress)
currentObservationImage=currentObservationHeader.open_memmap()
bgdatacube=np.array(currentObservationImage)
bgdatacube[bgdatacube==65535.0]=np.nan
outBand=imageFile.GetRasterBand(i + 1)
outBand.WriteArray(destinationDatacube[i,:,:])'''

In [ ]:
plt.imshow()

In [ ]:

plt.figure(figsize=(10,10),dpi=500)
plt.yticks(fontsize=35,visible=False)
plt.xticks(fontsize=35,visible=False)
plt.axis('off')
plt.box(False)

bothMap=np.where(co3Map>0,1,0)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((5,5)).astype(bothMap.dtype),iterations=1)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((2,2)).astype(bothMap.dtype),iterations=1)
maps=[]
for i in [13,78,233]:
    singleBand=datacube[:,:,i]
    singleBand=(singleBand-np.amin(singleBand[imageFootprint==1]))/np.ptp(singleBand[imageFootprint==1])
    singleBand[imageFootprint==-1]=np.amax(singleBand[imageFootprint==1])
    '''singleBand=np.where(imageFootprint>0,singleBand,np.amax(singleBand[np.where(~np.isnan(singleBand))]))
    #singleBand=singleBand[:,:]
    singleBand=Image.fromarray(singleBand)'''
    maps.append(singleBand)
grayScaleImage=np.dstack((maps))



plt.imshow(grayScaleImage)
overlay=np.where(bothMap!=1,np.nan,1)

overlay[0,0]=0

#plt.grid(visible=True,color='white')


plt.figure(figsize=(10,10),dpi=500)
plt.yticks(fontsize=35,visible=False)
plt.xticks(fontsize=35,visible=False)
#plt.yticks(fontsize=35,visible=True)
#plt.xticks(fontsize=35,visible=True)
plt.axis('off')
plt.box(False)

bothMap=np.where(co3Map>0,1,0)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((5,5)).astype(bothMap.dtype),iterations=1)
#bothMap=ndimage.median_filter(bothMap,size=5)

#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((2,2)).astype(bothMap.dtype),iterations=1)
maps=[]
for i in [13,13,13]:
    singleBand=datacube[:,:,i]
    singleBand=(singleBand-np.amin(singleBand[imageFootprint==1]))/np.ptp(singleBand[imageFootprint==1])
    singleBand[imageFootprint==-1]=np.amax(singleBand[imageFootprint==1])
    '''singleBand=np.where(imageFootprint>0,singleBand,np.amax(singleBand[np.where(~np.isnan(singleBand))]))
    #singleBand=singleBand[:,:]
    singleBand=Image.fromarray(singleBand)'''
    maps.append(singleBand)
grayScaleImage=np.dstack((maps))



plt.imshow(grayScaleImage)
overlay=np.where(bothMap!=1,np.nan,1)

overlay[0,0]=0

plt.imshow(overlay[:,:],cmap='rainbow')
#plt.grid(visible=True,color='white')


#calculate spectral parameters
MIN2295_2480,MIN2345_2537=calculateCarbonateBandDepthIndices(datacube,wavelengthList)


plt.figure(figsize=(10,10),dpi=500)
plt.yticks(fontsize=35,visible=False)
plt.xticks(fontsize=35,visible=False)
plt.axis('off')
plt.box(False)

bothMap=np.where(co3Map>0,1,0)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((5,5)).astype(bothMap.dtype),iterations=1)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((2,2)).astype(bothMap.dtype),iterations=1)
maps=[MIN2295_2480,MIN2345_2537,datacube[:,:,78]]

for i in range(len(maps)):
    singleBand=maps[i]
    singleBand=(singleBand-np.amin(singleBand[imageFootprint==1]))/np.ptp(singleBand[imageFootprint==1])
    singleBand[imageFootprint==-1]=np.amax(singleBand[imageFootprint==1])
    '''singleBand=np.where(imageFootprint>0,singleBand,np.amax(singleBand[np.where(~np.isnan(singleBand))]))
    #singleBand=singleBand[:,:]
    singleBand=Image.fromarray(singleBand)'''
    maps[i]=singleBand
grayScaleImage=np.dstack((maps))



plt.imshow(grayScaleImage)

In [ ]:
rowLow=150
rowHigh=200
colLow=500
colHigh=600


plt.figure(figsize=(10,10),dpi=500)
plt.yticks(fontsize=35,visible=False)
plt.xticks(fontsize=35,visible=False)
plt.axis('off')
plt.box(False)

bothMap=np.where(co3Map>0,1,0)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((5,5)).astype(bothMap.dtype),iterations=1)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((2,2)).astype(bothMap.dtype),iterations=1)
maps=[]
for i in [13,78,233]:
    singleBand=datacube[rowLow:rowHigh,colLow:colHigh,i]
    singleBand=(singleBand-np.amin(singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==1]))/np.ptp(singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==1])
    singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==-1]=np.amax(singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==1])
    '''singleBand=np.where(imageFootprint>0,singleBand,np.amax(singleBand[np.where(~np.isnan(singleBand))]))
    #singleBand=singleBand[:,:]
    singleBand=Image.fromarray(singleBand)'''
    maps.append(singleBand)
grayScaleImage=np.dstack((maps))



plt.imshow(grayScaleImage)
overlay=np.where(bothMap!=1,np.nan,1)

overlay[0,0]=0

plt.imshow(overlay[rowLow:rowHigh,colLow:colHigh],cmap='rainbow')
#plt.grid(visible=True,color='white')


plt.figure(figsize=(10,10),dpi=500)
#plt.yticks(fontsize=35,visible=False)
#plt.xticks(fontsize=35,visible=False)
plt.yticks(fontsize=35,visible=True)
plt.xticks(fontsize=35,visible=True)
#plt.axis('off')
#plt.box(False)

bothMap=np.where(co3Map>0,1,0)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((5,5)).astype(bothMap.dtype),iterations=1)
#bothMap=ndimage.median_filter(bothMap,size=5)
#bothMap=ndimage.morphology.binary_dilation(bothMap,structure=np.ones((2,2)).astype(bothMap.dtype),iterations=1)
maps=[]
for i in [13,13,13]:
    singleBand=datacube[rowLow:rowHigh,colLow:colHigh,i]
    singleBand=(singleBand-np.amin(singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==1]))/np.ptp(singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==1])
    singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==-1]=np.amax(singleBand[imageFootprint[rowLow:rowHigh,colLow:colHigh]==1])
    '''singleBand=np.where(imageFootprint>0,singleBand,np.amax(singleBand[np.where(~np.isnan(singleBand))]))
    #singleBand=singleBand[:,:]
    singleBand=Image.fromarray(singleBand)'''
    maps.append(singleBand)
grayScaleImage=np.dstack((maps))



plt.imshow(grayScaleImage)
overlay=np.where(bothMap!=1,np.nan,1)

overlay[0,0]=0

plt.imshow(overlay[rowLow:rowHigh,colLow:colHigh],cmap='rainbow')
plt.grid(visible=True,color='white')

In [ ]:
#plt.plot(wavelengthList,datacube[rowLow+27,colLow+42,:])
currentSpectra=datacube[rowLow+20,colLow+53,:]
currentSpectra=normalizeSpectra(currentSpectra)
plt.plot(wavelengthList,currentSpectra)
plt.plot(wavelengthList,plotHull(currentSpectra))

In [ ]:
#plt.plot(wavelengthList,datacube[rowLow+27,colLow+42,:])
currentSpectra=datacube[rowLow+20,colLow+53,:]
currentSpectra=normalizeSpectra(currentSpectra)
plt.plot(wavelengthList,currentSpectra)
plt.plot(wavelengthList,plotHull(currentSpectra))

In [ ]:
#set the address of the file where the denoised image is to be saved
inputImageAddress="G:/000093BE/FRT000093BE_07_IF166L_TRRB_sabcondpub_v1_b6_nr_ds_p_2014params.img"
#open the img file
imageFile=gdal.Open(inputImageAddress)
#read the datacube from the destination file
destinationDatacube=imageFile.ReadAsArray()
destinationDatacube[destinationDatacube==65535.0]=np.nan
outBand=imageFile.GetRasterBand(4)
outBand.WriteArray(bothMap)
del outBand
del destinationDatacube
del imageFile


In [ ]:
#create a list to hold all spectra collected for plotting
exemplarSpectras=[]
#create a list to hold indices of all collected spectra
exemplarPositions=[]

In [ ]:
spectraCollectionCube=ndimage.median_filter(bgdatacube,footprint=np.ones((3,3,1)),mode='reflect')

In [ ]:
rowLow=700
rowHigh=580
colLow=200
colHigh=300

plt.figure(figsize=(10,10))

maps=[]
for i in [13,14,15]:
    singleBand=bgdatacube[:,:,i]
    singleBand=np.where(imageFootprint>0,singleBand,np.max(singleBand))
    singleBand=singleBand[:,:]
    singleBand=Image.fromarray(singleBand)
    maps.append(singleBand)
grayScaleImage=np.dstack((maps))
plt.imshow(grayScaleImage[rowLow:,colLow:colHigh,:])
overlay=np.where(bothMap!=1,np.nan,1)

overlay[0,0]=0

plt.imshow(overlay[rowLow:,colLow:colHigh],cmap='rainbow')
#plt.scatter([55],[55],s=20)
plt.grid(visible=True,color='white')
bgdatacube
currentSpectra=spectraCollectionCube[rowLow+20,colLow+60,:]
currentSpectra=datacube[rowLow+24,colLow+31,:]
currentSpectra=normalizeSpectra(currentSpectra)
'''exemplarSpectras.append(currentSpectra)
exemplarPositions.append([rowLow+5,colLow+10])
currentSpectra=datacube[rowLow+35,colLow+37,:]
exemplarSpectras.append(currentSpectra)
exemplarPositions.append([rowLow+35,colLow+37])'''
plt.figure(figsize=(30,15))
plt.plot(wavelengthList,currentSpectra,lw=3)
plt.plot(wavelengthList,plotHull(currentSpectra),lw=3)

In [ ]:
#calculate spectral parameters
MIN2295_2480,MIN2345_2537=calculateCarbonateBandDepthIndices(datacube,wavelengthList)
#threshold the arrays

MIN2295_2480=np.where(imageFootprint>0,MIN2295_2480,np.nan)
MIN2345_2537=np.where(imageFootprint>0,MIN2345_2537,np.nan)
#get unified map of standard parameters
originalCO3Map=np.maximum(MIN2295_2480,MIN2345_2537)
originalCO3Map=np.where(originalCO3Map>0.03,1,0)

plt.figure(figsize=(10,10))
plt.yticks(fontsize=35,visible=False)
plt.xticks(fontsize=35,visible=False)
plt.axis('off')
plt.box(False)

maps=[]
for i in [13,14,15]:
    singleBand=datacube[:,:,i]
    singleBand=np.where(imageFootprint>0,singleBand,1)
    singleBand=singleBand[:,:]
    singleBand=Image.fromarray(singleBand)
    maps.append(singleBand)
grayScaleImage=np.dstack((maps))
plt.imshow(grayScaleImage)
overlay=np.where(originalCO3Map!=1,np.nan,1)

overlay[0,0]=0
print(np.max(singleBand))
plt.imshow(overlay[:,:],cmap='rainbow')


In [ ]:
#create figure
plt.figure(figsize=(27,25))
plt.title("",fontsize=25)
plt.yticks(fontsize=35,visible=False)
plt.xticks(fontsize=35,visible=True)
plt.ylabel("CRISM I/F offset",fontsize=35)
plt.xlabel("Wavelength in micron",fontsize=35)
#create variable to provide offset in plotting spectra
offsets=[0,0.2,0.25,0.15,0.25,0.1]
offset=0
#define minimum band depth for a feature to be considered
minimumFeatureWidth=0.05
#iterate over all spectras
for i in range(len(exemplarPositions)):
    #get spectra and its indices
    currentSpectra=exemplarSpectras[i]
    currentCoordinates=exemplarPositions[i]
    #update offset
    offset=offset+offsets[i]
    #plot the spectrum
    plt.plot(wavelengthList,currentSpectra-offset,label="("+str(currentCoordinates[0])+","+str(currentCoordinates[1])+")",lw=5)
    '''#apply continuum removal and get band centres 
    cr=plotHull(currentSpectra)
    features,wavelengths=getAbsorptionFeatures(cr)
    tempFeat=[]
    tempWavelengths=[]
    for i in range(len(wavelengths)):
        if wavelengths[i][-1]-wavelengths[i][0]>minimumFeatureWidth:
            tempFeat.append(features[i])
            tempWavelengths.append(wavelengths[i])
    features=tempFeat
    wavelengths=tempWavelengths
    bandCentres,bandDepths,fwhms,leftShoulders,rightShoulders=getFeatureParameters(wavelengths,features)
    #create variables to get reflectances and band centres for 2.3 and 2.5 micron features
    bandCentre23=bandCentre25=reflectance23=reflectance25=0
    #iterate over all features
    for j in range(len(bandCentres)):
        #get current band centre
        currentBandCentre=bandCentres[j]
        #plot only 2.3 micron feature
        if currentBandCentre>2.25 and currentBandCentre<2.35:
            #get band index corresponding to this band centre and reflectance there
            bandIndex=find_nearest_index(wavelengthList,currentBandCentre)
            currentReflectance=currentSpectra[bandIndex]
            plt.scatter(currentBandCentre,currentReflectance-offset,color='black',s=300)
            bandCentre23=currentBandCentre
            reflectance23=currentReflectance
            
        #plot only 2.5 micron feature
        if currentBandCentre>2.48 and currentBandCentre<2.6:
            #get band index corresponding to this band centre and reflectance there
            bandIndex=find_nearest_index(wavelengthList,currentBandCentre)
            currentReflectance=currentSpectra[bandIndex]
            plt.scatter(currentBandCentre,currentReflectance-offset,color='black',s=300)
            bandCentre25=currentBandCentre
            reflectance25=currentReflectance
        
        #add the band-centre values as text to the plot and add inter band-gap
        if bandCentre23!=0 and bandCentre25!=0:
            plt.text(bandCentre23+0.02,reflectance23-offset,str(round(bandCentre23,3)),fontsize=25)
            plt.text(bandCentre25+0.02,reflectance25-offset-0.05,str(round(bandCentre25,3)),fontsize=25)

            plt.vlines(x=bandCentre23,ymin=reflectance25-offset,ymax=reflectance23-offset,color='black',lw=2,ls='dotted')
            plt.hlines(y=reflectance25-offset,xmin=bandCentre23,xmax=bandCentre25,color='black',lw=2,ls='dashed')
            
            plt.text((bandCentre25+bandCentre23)/2-0.05,reflectance25-offset+0.00,str(round(bandCentre25-bandCentre23,3)),fontsize=25,color='brown')'''
#plt.legend(loc="best",framealpha=1,fontsize=55)
plt.vlines(x=2.30,
           ymin=min(exemplarSpectras[-1])-offset,
           ymax=np.amax(np.array(exemplarSpectras)),
           lw=2,color='black')
plt.vlines(x=2.50,
           ymin=min(exemplarSpectras[-1])-offset,
           ymax=np.amax(np.array(exemplarSpectras)),
           lw=2,color='black')
plt.text(2.30,
         np.amax(np.array(exemplarSpectras)),
         '2.30',
         color='black',
         fontsize=35)
plt.text(2.50,
         np.amax(np.array(exemplarSpectras)),
         '2.50',
         color='black',
         fontsize=35)